In [1]:
import numpy as np
import pandas as pd
import os
#from osgeo import gdal
import geopandas as gpd
#import rasterio
import osmnx as ox
import rioxarray
import pyproj
import shapely.geometry as sg
import networkx as nx
import momepy

In [2]:
# Test data
test_path = "C:/Users/ygrin/Documents/Studie - MSc ADS/Utrecht University/Block 4 - Thesis/TestData/"
single_point = gpd.read_file(test_path + "Test_single_home_location.gpkg")
multi_point = gpd.read_file(test_path + "Test_multiple_home_locations.gpkg")
#polygon = gpd.read_file(test_path + "TestArea.gpkg")
ndvi = rioxarray.open_rasterio(test_path + "NDVI_data_test.tif")
land_cover = rioxarray.open_rasterio(test_path + "Landcover_data_test.tif")
#network = gpd.read_file(test_path + "test_network.gpkg", layer='edges')
network = gpd.read_file(test_path + "test_network_shp.shp")
polygon = gpd.read_file(test_path + "test_polygon.gpkg")
canopy_single = gpd.read_file(test_path + "Canopy_single_home_500m.gpkg")

single_point_geographic = single_point.to_crs('epsg:4326')

# Greenness Accessibility -- Shortest Network Distance to Park

In [3]:
def calculate_shortest_distance(geom=None, buffer_dist=None, network_graph=None, park_src=None):   
    park_buffer = park_src.clip(geom.buffer(buffer_dist))

    # get nearest node to house location
    nearest_node = ox.distance.nearest_nodes(network_graph, geom.x, geom.y)
    # Create dictionary to extract geometries for nodes of interest later on
    pos = {n: (network_graph.nodes[n]['x'], network_graph.nodes[n]['y']) for n in network_graph.nodes}
    # get nodes within 25 meters of clipped park polygon boundaries
    boundary_nodes = []
    for boundary in park_buffer.boundary:
        for node in network_graph.nodes():
            node_pos = sg.Point(pos[node])
            if node_pos.distance(boundary) < 25:
                boundary_nodes.append(node)

    # calculate network distances from house location's nearest node to nodes closest to park boundaries within buffer distance from house
    distances = {}
    for node in boundary_nodes:
        try:
            path = nx.shortest_path(network_graph, nearest_node, node, weight='length') 
            distance = sum([network_graph.edges[path[i], path[i+1], 0]['length'] for i in range(len(path)-1)])
            distances[node] = distance
        except:
            pass

    # get minimum distance
    if distances:
        min_distance = min(distances.values())
    else:
        min_distance = np.nan
    
    return min_distance

In [5]:
def get_shortest_distance_park(point_of_interest_file, buffer_dist=500, park_vector_file=None, network_file=None, network_type=None, 
                               output_dir=os.getcwd()):
    # Read and process user input, check conditions
    poi = gpd.read_file(point_of_interest_file)
    if all(poi['geometry'].geom_type == 'Point') or all(poi['geometry'].geom_type == 'Polygon'):
        geom_type = poi.iloc[0]['geometry'].geom_type
    else:
        raise ValueError("Please make sure all geometries are of 'Point' type or all geometries are of 'Polygon' type and re-run the function")

    if not poi.crs.is_projected:
        raise ValueError("The CRS of the PoI dataset is currently geographic, please transform it into a local projected CRS and re-run the function")
    else:
        epsg = poi.crs.to_epsg()

    # In case of house polygons, transform to centroids
    if geom_type == "Polygon":
        print("Changing geometry type to Point by computing polygon centroids so that network distance can be retrieved...")
        poi['geometry'] = poi['geometry'].centroid
        print("Done \n")

    if "id" in poi.columns:
        if poi['id'].isnull().values.any():
            poi['id'] = poi['id'].fillna(pd.Series(range(1, len(poi) + 1))).astype(int)
    else:
        poi['id'] = pd.Series(range(1, len(poi) + 1)).astype(int)

    if not isinstance(buffer_dist, int) or (not buffer_dist > 0):
        raise TypeError("Please make sure that the buffer distance is set as a positive integer")

    epsg_transformer = pyproj.Transformer.from_crs(f"epsg:{epsg}", "epsg:4326") # EPSG transformer to use for OSM            
    # Read park polygons, retrieve from OSM if not provided by user 
    if park_vector_file is not None:
        park_src = gpd.read_file(park_vector_file)
        if not park_src.crs.to_epsg() == epsg:
            print("Adjusting CRS of Park file to match with Point of Interest CRS...")
            park_src.to_crs(f'EPSG:{epsg}', inplace=True)
            print("Done \n")
    else:
        print("Retrieving parks within buffer distance for point(s) of interest...")
        park_tags = {'leisure': 'park', 'boundary': 'national_park', 'landuse': 'recreation_ground'}
        park_src = gpd.GeoDataFrame()
        for geom in poi['geometry']:
            latlon = epsg_transformer.transform(geom.x, geom.y)
            park_geom = ox.geometries_from_point(latlon, tags=park_tags, dist=buffer_dist)
            park_src = gpd.GeoDataFrame(pd.concat([park_src, park_geom], ignore_index=True), crs=park_geom.crs)
        park_src.to_crs(f"EPSG:{epsg}", inplace=True)
        print("Done \n")

    # Read road network, retrieve from OSM if not provided by user 
    if network_file is not None:
        if os.path.splitext(network_file)[1] not in [".gpkg", ".shp"]:
            raise ValueError("Please provide the network file in '.gpkg' or '.shp' format")
        elif network_file is not None and (os.path.splitext(network_file)[1] == ".gpkg"):
            network = gpd.read_file(network_file, layer='edges')
        elif network_file is not None and (os.path.splitext(network_file)[1] == ".shp"):
            network = gpd.read_file(network_file)

        if not network.crs.to_epsg() == epsg:
            print("Adjusting CRS of Network file to match with Point of Interest CRS...")
            network.to_crs(f'EPSG:{epsg}', inplace=True)
            print("Done \n")

        # Check if house locations are within network file provided
        bbox_network = network.unary_union.envelope
        if not all(geom.within(bbox_network) for geom in poi['geometry']):
            raise ValueError("Not all points of interest are within the network file provided, please make sure they are and re-run the function")

        # Convert network to graph object using momempy
        network_graph = momepy.gdf_to_nx(network).to_directed()
    else:
        if network_type not in ["walk", "bike", "drive", "all"]:
            raise ValueError("Please make sure that the network_type argument is set to either 'walk', 'bike, 'drive' or 'all', and re-run the function")

        print("Retrieving infrastructure network within buffer distance for point(s) of interest...")
        graph_list = []
        for geom in poi['geometry']:
            latlon = epsg_transformer.transform(geom.x, geom.y)
            network = ox.graph_from_point(latlon, dist=buffer_dist, network_type=network_type)
            network = ox.project_graph(network, to_crs=f"EPSG:{epsg}")
            graph_list.append(network)

        network_graph = nx.MultiDiGraph()
        for graph in graph_list:
            network_graph = nx.union(network_graph, graph)
        print("Done \n")
    
    print("Calculating shortest distances...")
    poi['shortest_distance_park'] = poi.geometry.apply(lambda x: calculate_shortest_distance(geom=x, buffer_dist=buffer_dist, network_graph=network_graph, park_src=park_src))
    print("Done \n")
    
    print("Writing results to new geopackage file in specified directory...")
    input_filename, ext = os.path.splitext(point_of_interest_file)
    poi.to_file(os.path.join(output_dir, f"{input_filename}_ShortDistPark_added.gpkg"), driver="GPKG")
    print("Done")

    return poi

In [256]:
shortest_dist = get_shortest_distance_park(point_of_interest_file=test_path + "Test_multiple_home_locations.gpkg", buffer_dist=500, 
                                           park_vector_file=None, network_file=None, network_type='bike', output_dir=os.getcwd())

Retrieving parks within buffer distance for point(s) of interest...
Done 

Retrieving infrastructure network within buffer distance for point(s) of interest...
Done 

Calculating shortest distances...


c:\Users\ygrin\anaconda3\envs\spatial\lib\site-packages\geopandas\tools\clip.py:67: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  clipped.loc[
c:\Users\ygrin\anaconda3\envs\spatial\lib\site-packages\geopandas\tools\clip.py:67: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  clipped.loc[
c:\Users\ygrin\anaconda3\envs\spatial\lib\site-packages\geopandas\tools\clip.py:67: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use

Done 

Writing results to new geopackage file in specified directory...
Done


In [257]:
shortest_dist

,id,geometry,shortest_distance_park
0,1,POINT (388644.249 392861.634),346.715
1,2,POINT (385981.911 393805.494),456.385
2,3,POINT (388631.230 395322.181),510.566


# Testing

In [7]:
shortest_dist = get_shortest_distance_park(point_of_interest_file=test_path + "Test_single_home_location.gpkg", buffer_dist=500, 
                                           park_vector_file=None, network_file=test_path + "test_network.gpkg", network_type=None, 
                                           output_dir=os.getcwd())

Retrieving parks within buffer distance for point(s) of interest...
Done 

Adjusting CRS of Network file to match with Point of Interest CRS...
Done 

Calculating shortest distances...


TypeError: Must pass list-like as `names`.

In [66]:
test_network = gpd.read_file(test_path + "test_network_shp.shp").to_crs("EPSG:27700")
test_network_graph = momepy.gdf_to_nx(test_network).to_directed()

In [33]:
epsg_transformer = pyproj.Transformer.from_crs("epsg:27700", "epsg:4326")
latlon = epsg_transformer.transform(single_point['geometry'].x, single_point['geometry'].y) # Transform point geometry into latlon for OSMnx
network_osmnx = ox.graph_from_point(latlon, dist=500, network_type='walk')